# **Model Training**

In [1]:
from os.path import join
from google.colab import drive
# Montar el DRIVE
ROOT = "/content/drive"
drive.mount(ROOT)# Apuntar a la carpeta de trabajo
PROJ = "My Drive/output" # This is a custom path.
#PROJECT_PATH = join(ROOT, PROJ)


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR, NuSVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor,ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [ ]:
X = pd.read_csv(join(PROJECT_PATH,'X.csv'))
y = pd.read_csv(join(PROJECT_PATH,'y.csv'))

In [ ]:
X = X.drop(columns="Unnamed: 0")
y = y.drop(columns="Unnamed: 0")
y = y["price"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

**This is a regression issue** --> Calculate a price based on diamond characteristics.
Therefore, train different algorithms for regressions

In [11]:
models = {
    "RegresionLineal" : LinearRegression(),
    "Ridge": Ridge(alpha=.5),
    "Lasso": Lasso(alpha=0.1),
    "Bayesian": BayesianRidge(),
    "SVR": SVR(),
    "KNeighbors": KNeighborsRegressor(),
    "RadiusNeighbors": RadiusNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "ExtraTree": ExtraTreeRegressor(),
    "Dummy": DummyRegressor(),
    "Ada Boost": AdaBoostRegressor(),
    "Bagging": BaggingRegressor(),
    "ExtraTrees": ExtraTreesRegressor(),
    "GradientBoosting" : GradientBoostingRegressor(),
    "RandomForest" : RandomForestRegressor(),
    "HistGradientBoost": HistGradientBoostingRegressor(),
    
    }

for modelName, model in models.items():
    print(f"Training model: {modelName}")
    model.fit(X_train, y_train)

Training model: RegresionLineal
Training model: Ridge
Training model: Lasso


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17628450487.033882, tolerance: 51103061.05889814
  positive)


Training model: Bayesian
Training model: SVR
Training model: KNeighbors
Training model: RadiusNeighbors
Training model: DecisionTree
Training model: ExtraTree
Training model: Dummy
Training model: Ada Boost
Training model: Bagging
Training model: ExtraTrees
Training model: GradientBoosting
Training model: RandomForest
Training model: HistGradientBoost


**Display predicted prices by all trained models**

In [ ]:
d = {modelName:model.predict(X_test) for modelName, model in models.items()}
df = pd.DataFrame(d)
df["gt"] = y_test.reset_index(drop=True)
df

,RegresionLineal,Ridge,Lasso,Bayesian,SVR,KNeighbors,RadiusNeighbors,DecisionTree,ExtraTree,Dummy,Ada Boost,Bagging,ExtraTrees,GradientBoosting,RandomForest,HistGradientBoost,gt
0,6350.561854,6352.386157,6346.601937,6350.690390,4219.189465,6525.4,5953.232877,5777.0,6488.0,3925.105487,6718.052820,6447.1,6314.44,6671.114690,6511.600,6630.721107,6310
1,355.545813,354.165330,355.506214,355.448910,1174.209822,681.4,858.191011,667.0,667.0,3925.105487,1135.563442,663.8,674.92,820.938713,680.170,778.640959,680
2,7556.419994,7556.343138,7553.923400,7556.414791,5111.511583,5507.4,5702.700000,5358.0,5834.0,3925.105487,6883.983077,5602.6,5945.89,5280.344451,5823.810,6036.205479,6544
3,1553.565912,1556.187901,1549.462386,1553.750604,1628.163525,981.8,1123.545455,953.0,990.0,3925.105487,1135.563442,932.7,999.72,819.569602,966.240,962.209351,926
4,-1220.868786,-1226.081049,-1222.870978,-1221.235706,934.218325,473.0,499.047619,436.0,432.0,3925.105487,1135.563442,434.0,453.16,348.179116,441.330,455.157714,432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8061,6123.707876,6123.430329,6119.825082,6123.688315,3853.971154,5138.8,4665.571429,4977.0,4977.0,3925.105487,6226.425321,5386.0,5153.07,5353.126543,5249.830,5260.530539,5443
8062,6114.034314,6115.562255,6116.764285,6114.141697,4349.191225,5162.6,5094.538462,4813.0,5226.0,3925.105487,6226.425321,4952.6,5086.69,5364.543517,5015.380,5245.611352,4901
8063,571.298704,567.523244,572.041458,571.032642,801.885414,744.6,759.553398,802.0,828.0,3925.105487,1135.563442,786.9,769.73,683.521977,783.220,736.373646,828
8064,9529.555134,9526.223465,9526.407062,9529.320692,5647.987819,7954.6,7827.675000,8637.0,7936.0,3925.105487,10994.643241,8101.3,8398.05,9318.726940,8201.245,8211.711023,10543


**Display scores "mse", "rmse" and "R²" for predictions**

In [ ]:
score_models ={"Model":[],"mse":[],"rmse":[],"R²":[]}
for modelName, model in models.items():   
    score_models["Model"].append(modelName) 
    score_models["mse"].append(mean_squared_error(df["gt"], df[modelName]))
    score_models["rmse"].append(sqrt(mean_squared_error(df["gt"], df[modelName])))
    score_models["R²"].append(r2_score(df["gt"], df[modelName]))

In [ ]:
score_models_df = pd.DataFrame(score_models)
display(score_models_df)
min_rmse = list(score_models_df["rmse"].sort_values()[:3].values)
min_rmse
print("The 3 best models based on rmse score are:")
for e in min_rmse:
  display(score_models_df["Model"][score_models_df["rmse"] == e].values[0])

,Model,mse,rmse,R²
0,RegresionLineal,1.460930e+06,1.208689e+03,9.046062e-01
1,Ridge,1.461673e+06,1.208997e+03,9.045577e-01
2,Lasso,1.461465e+06,1.208911e+03,9.045713e-01
3,Bayesian,1.460981e+06,1.208710e+03,9.046029e-01
4,SVR,7.424308e+06,2.724758e+03,5.152179e-01
5,KNeighbors,3.993028e+05,6.319041e+02,9.739269e-01
6,RadiusNeighbors,2.109363e+35,4.592780e+17,-1.377342e+28
7,DecisionTree,5.263000e+05,7.254653e+02,9.656344e-01
8,ExtraTree,5.619830e+05,7.496553e+02,9.633044e-01
9,Dummy,1.531500e+07,3.913439e+03,-1.782374e-05


The 3 best models based on rmse score are:


'HistGradientBoost'

'ExtraTrees'

'RandomForest'

The 3 best rmse scores have been achieved by Ensemble models. 
Optimize parameters for these 3 models: "RandomForestRegressor", "HistGradientBoostingRegressor" and "ExtraTreesRegressor"

### **Optimize parameters**

In [11]:
def gridSearch(algorithm, params):
    tuning_gradient = GridSearchCV(estimator = algorithm,param_grid=params, scoring='neg_mean_squared_error',iid=False, cv=5, verbose=1)
    tuning_gradient.fit(X_train,y_train)
    return tuning_gradient.best_params_


params = {'n_estimators':[100, 200, 500, 1000],
          'max_features':[1,2,3,4,5,6,7,8]}

gridSearch(RandomForestRegressor(min_samples_split=2, min_samples_leaf=1, bootstrap = True), params)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed: 80.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'max_features': 5, 'n_estimators': 500}

In [13]:
params = {'learning_rate':[0.15,0.1,0.05],
          'max_iter':[100, 500, 1000]}

gridSearch(HistGradientBoostingRegressor(loss='least_squares', n_iter_no_change=None, max_leaf_nodes=31, max_depth=None, min_samples_leaf=20), params)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'learning_rate': 0.05, 'max_iter': 500}

In [14]:
params = {'learning_rate':[0.02,0.05, 0.075, 0.1],
          'max_iter':[200, 500, 800]}

gridSearch(HistGradientBoostingRegressor(loss='least_squares', n_iter_no_change=None, max_leaf_nodes=31, max_depth=None, min_samples_leaf=20), params)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  2.7min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'learning_rate': 0.02, 'max_iter': 800}

In [15]:
params = {'learning_rate':[0.005, 0.01, 0.02,0.05],
          'max_iter':[400, 500, 800, 1000]}

gridSearch(HistGradientBoostingRegressor(loss='least_squares', n_iter_no_change=None, max_leaf_nodes=31, max_depth=None, min_samples_leaf=20), params)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  5.3min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'learning_rate': 0.02, 'max_iter': 800}

In [19]:
params = {'n_estimators':[100, 500, 1000],
          'max_features':[4,5,6,7]}

gridSearch(ExtraTreesRegressor(min_samples_split=2, min_samples_leaf=1, bootstrap=True), params)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 17.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'max_features': 7, 'n_estimators': 1000}

In [20]:
models = {
    "ExtraTrees": ExtraTreesRegressor(min_samples_split=2, min_samples_leaf=1, bootstrap=True, max_features = 7, n_estimators=1000),
    "RandomForest" : RandomForestRegressor(min_samples_split=2, min_samples_leaf=1, bootstrap = True, max_features = 5, n_estimators = 500),
    "HistGradientBoost": HistGradientBoostingRegressor(loss='least_squares', n_iter_no_change=None, max_leaf_nodes=31, max_depth=None, min_samples_leaf=20,learning_rate=0.02, max_iter=800),
    }

for modelName, model in models.items():
    print(f"Training model: {modelName}")
    model.fit(X_train, y_train)

Training model: ExtraTrees
Training model: RandomForest
Training model: HistGradientBoost


In [8]:
d = {modelName:model.predict(X_test) for modelName, model in models.items()}
df = pd.DataFrame(d)
df["gt"] = y_test.reset_index(drop=True)
score_models ={"Model":[],"mse":[],"rmse":[],"R²":[]}
for modelName, model in models.items():   
    score_models["Model"].append(modelName) 
    score_models["mse"].append(mean_squared_error(df["gt"], df[modelName]))
    score_models["rmse"].append(sqrt(mean_squared_error(df["gt"], df[modelName])))
    score_models["R²"].append(r2_score(df["gt"], df[modelName]))
score_models_df = pd.DataFrame(score_models)
score_models_df

,Model,mse,rmse,R²
0,ExtraTrees,302704.712360,550.186071,0.980860
1,RandomForest,310191.418715,556.948309,0.980386
2,HistGradientBoost,297980.407897,545.875817,0.981158


**After submitting these 3 models, the best score has been achieved with Extra Tress Regressor model. Continue improving this model**

In [23]:
params = {'n_estimators':[700, 1000, 1500],
          'max_features':[5,6,7,8]}

gridSearch(ExtraTreesRegressor(min_samples_split=2, min_samples_leaf=1, bootstrap=True), params)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 37.3min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


GridSearchCV(cv=5, error_score=nan,
             estimator=ExtraTreesRegressor(bootstrap=True, ccp_alpha=0.0,
                                           criterion='mse', max_depth=None,
                                           max_features='auto',
                                           max_leaf_nodes=None,
                                           max_samples=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=100, n_jobs=None,
                                           oob_score=False, random_state=None,
                                           verbose=0, warm_start=False),
             iid=False, n_jobs=None,
             param_grid=

In [24]:
tuning_Trees.best_params_

{'max_features': 6, 'n_estimators': 1000}

In [25]:
params = {'max_features':[6,7],
          'random_state':[None,1,5,10]}
          
gridSearch(ExtraTreesRegressor(min_samples_split=2, min_samples_leaf=1, bootstrap=True, n_estimators = 1000), params)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 23.7min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'max_features': 7, 'random_state': None}

This last search for optimizing parameters has not resulted in any improvement.

### **Make predictions on test dataset and save data for submission**

In [ ]:
test = pd.read_csv(join(PROJECT_PATH,'test.csv'))
test.drop(columns="Unnamed: 0",inplace=True)

In [18]:
model = ExtraTreesRegressor(min_samples_split=2, min_samples_leaf=1, bootstrap=True, max_features = 7, n_estimators=1000)
model.fit(X, y)

ExtraTreesRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features=7, max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=1000, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred = model.predict(test)

In [ ]:
y_pred = pd.DataFrame(y_pred, columns=["price"])
y_pred = y_pred.reset_index()
y_pred = y_pred.rename(columns={'index':'id'})

In [21]:
y_pred

,id,price
0,0,426.7540
1,1,1969.7940
2,2,9473.2070
3,3,497.9594
4,4,9258.9910
...,...,...
13444,13444,4749.1500
13445,13445,445.2950
13446,13446,14977.7610
13447,13447,10428.3530


In [ ]:
y_pred.to_csv(join(PROJECT_PATH,'submission.csv'), index=False)